# Neural Network for racetime prediction

### Define a logger for logging experiments configurations and results

In [11]:
import json
import os
from datetime import datetime
from pytorch_lightning.loggers import Logger
from pytorch_lightning.utilities.rank_zero import rank_zero_only

class F1NeuralNetworkExperimentsLogger(Logger):
    def __init__(self):
        super().__init__()
        self.metrics = []
        self.logs = {}
        self._version = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    @property
    def name(self):
        return "F1NeuralNetworkExperimentsLogger"

    @property
    def version(self):
        return self._version
    
    @rank_zero_only
    def log_model_architecture(self, model):
        def layer_to_dict(layer):
            """Convert a PyTorch layer to a structured dictionary."""
            return {
                'type': layer.__class__.__name__,
                'parameters': {
                    name: p.shape if hasattr(p, 'shape') else str(p)
                    for name, p in layer.named_parameters(recurse=False)
                },
                'submodules': [layer_to_dict(sub) for sub in layer.children()]
            }

        architecture_dict = layer_to_dict(model)
        self.logs['model_architecture'] = architecture_dict


    @rank_zero_only
    def log_hyperparams(self, params):
        self.logs['hyperparameters'] = {k: str(v) for k, v in params.items()}

    @rank_zero_only
    def log_metrics(self, metrics, step):
        self.metrics.append((step, metrics))
        if 'metrics' not in self.logs:
            self.logs['metrics'] = []
        self.logs['metrics'].append({'step': step, 'metrics': {k: float(v) for k, v in metrics.items()}})

    @rank_zero_only
    def log_overall_test_loss(self, test_loss):
        self.logs['overall_test_loss'] = test_loss

    @rank_zero_only
    def log_used_features(self, features):
        self.logs['used_features'] = features

    @rank_zero_only
    def log_optimization_strategy(self, optimizer, scheduler):
        optimizer_str = str(optimizer)
        scheduler_str = str(scheduler)
        self.logs['optimizer'] = optimizer_str
        self.logs['scheduler'] = scheduler_str

    @rank_zero_only
    def save(self):
        directory = os.path.join(self.name, self.version)
        if not os.path.exists(directory):
            os.makedirs(directory)
        with open(os.path.join(directory, "logs.json"), "w") as f:
            json.dump(self.logs, f, indent=4)

    @rank_zero_only
    def finalize(self, status):
        self.save()

logger = F1NeuralNetworkExperimentsLogger()

### Load the features and labels for regression

In [12]:
from data_preparation import load_and_preprocess_data, prepare_regression_data, split_data_by_race
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class F1Dataset(Dataset):
    def __init__(self, X, y, scaler=None):
        super().__init__()
        self.X = X.apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
        self.y = y.reset_index(drop=True)
        self.scaler = scaler
        if self.scaler:
            self.X = self.scaler.transform(self.X)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(self.y.iloc[idx], dtype=torch.float32)
        return X, y

train_df, test_df = split_data_by_race(load_and_preprocess_data())
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)

X_train, y_train = prepare_regression_data(train_df)
X_val, y_val = prepare_regression_data(val_df)
X_test, y_test = prepare_regression_data(test_df)

N_FEATURES = X_train.shape[1]

train_dataset = F1Dataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = F1Dataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=32)
test_dataset = F1Dataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32)

train_dataset.__getitem__(0)

/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/data_preparation.py:25: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  practice_sessions = pd.read_csv('../data/raw_data/ff1_laps.csv', na_values=na_values)
/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/data_preparation.py:26: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  tire_data = pd.read_csv('../data/raw_data/ff1_laps.csv', na_values=na_values)


(586171, 15)
(586171, 32)
(586171, 40)
(586171, 45)
(586171, 46)
(586171, 47)
(586171, 47)


/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/data_preparation.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  laps['pitstop_milliseconds'].fillna(0, inplace=True)  # Assuming 0 if no pit stop
/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/data_preparation.py:106: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace meth

(586171, 56)


/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/data_preparation.py:196: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  laps['Compound'].fillna('UNKNOWN', inplace=True)
/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/data_preparation.py:249: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

Matching for 1
Matching for 2
Matching for 3
Matching for 4
Matching for 5
Matching for 6
Matching for 7
Matching for 8
Matching for 9
Matching for 10
Matching for 11
Matching for 12
Matching for 13
Matching for 14
Matching for 15
Matching for 16
Matching for 17
Matching for 18
Matching for 19
Matching for 20
Matching for 21
Matching for 22
Matching for 23
Matching for 24
Matching for 25
Matching for 26
Matching for 27
Matching for 28
Matching for 29
Matching for 30
Matching for 31
Matching for 32
Matching for 33
Matching for 34
Matching for 35
Matching for 36
Matching for 37
Matching for 38
Matching for 39
Matching for 40
Matching for 41
Matching for 42
Matching for 43
Matching for 44
Matching for 45
Matching for 46
Matching for 47
Matching for 48
Matching for 49
Matching for 50
Matching for 51
Matching for 52
Matching for 53
Matching for 54
Matching for 55
Matching for 56
Matching for 57
Matching for 58
Matching for 59
Matching for 60
Matching for 61
Matching for 62
Matching for 63
M

/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/data_preparation.py:492: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  laps['TrackStatus'].fillna(1, inplace=True)  # 1 = regular racing status


Shape before filtering and outlier removal: (586171, 95)
Normal racing laps: (528281, 95)
Special laps (pit stops, safety car, etc.): (57890, 95)
Final shape after outlier removal: (555735, 95)


(tensor([4.1873e-01, 4.1873e-01, 5.0000e-01, 5.0000e-01, 2.1115e-01, 2.1115e-01,
         7.8018e+04, 8.0018e+04, 8.7219e+04, 8.2268e+04, 2.0000e+01, 5.0000e+01,
         5.0000e+00, 2.5000e+01, 2.0000e+01, 5.0000e+01, 1.0000e+00, 1.0000e+00,
         0.0000e+00]),
 tensor(76849.))

### Model

In [13]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import matplotlib.pyplot as plt

class LaptimePredicionModel(pl.LightningModule):
    def __init__(self, lr=1e-3, loss_fn=nn.L1Loss, optimizer=torch.optim.AdamW, layer_config=(3, 64)):
        super().__init__()
        self.save_hyperparameters(logger=True)
        self.predictions = []
        self.actuals = []
        self.train_losses = []
        self.val_losses = []
        self.test_losses = []
        self.lr = lr
        self.INPUT_DIM = N_FEATURES
        self.OUTPUT_DIM = 1

        # Get the number of layers and neurons per layer from layer_config
        num_layers, neurons_per_layer = layer_config
        
        # Build the model dynamically
        layers = []
        
        in_features = self.INPUT_DIM
        for i in range(num_layers):
            layers.append(nn.Linear(in_features, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            in_features = neurons_per_layer
        
        # Final output layer
        layers.append(nn.Linear(neurons_per_layer, self.OUTPUT_DIM))

        self.model = nn.Sequential(*layers)
        self.loss_fn = loss_fn()
        self.optimizer = optimizer(self.model.parameters(), lr=self.lr)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(-1)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.train_losses.append(loss.item())
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(-1)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.val_losses.append(loss.item())
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(-1)
        test_loss_fn = nn.MSELoss()
        loss = torch.sqrt(test_loss_fn(y_hat, y))
        self.predictions.extend(y_hat.cpu().numpy())
        self.actuals.extend(y.cpu().numpy())
        self.test_losses.append(loss.item())
        return loss

    def on_test_epoch_end(self):
        avg_test_loss = sum(self.test_losses) / len(self.test_losses)
        print(f"Average Test Loss: {avg_test_loss}")
        
        # Plot the correlation between actual and predicted values
        plt.figure(figsize=(8, 6))
        plt.scatter(self.actuals, self.predictions, alpha=0.6, edgecolor='k')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title('Correlation between Actual and Predicted Values')
        plt.grid(True)
        plt.show()
        
        self.plot_loss_curve()

    def configure_optimizers(self):
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=self.hparams.lr,
            steps_per_epoch=100,
            epochs=10,
            anneal_strategy='cos',
        )
        return {
            'optimizer': self.optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

    def plot_loss_curve(self):
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_losses, label='Training Loss')
        if self.val_losses:
            plt.plot(self.val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss Curve')
        plt.legend()
        plt.grid(True)
        plt.show()

### Configure callback functions

In [14]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,          
    verbose=True,
    mode="min"           
)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",     
    filename="best_model-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,             
    mode="min",                  
    verbose=True
)

### Determine optimal amount of model layers and neurons per layer

In [ ]:
import pytorch_lightning as pl
import optuna

def objective(trial):
    num_layers = trial.suggest_int('num_layers', 2, 6)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 32, 256)

    model = LaptimePredicionModel(layer_config=(num_layers, neurons_per_layer))

    trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=[early_stopping, checkpoint_callback])

    trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

    avg_val_loss = sum(model.val_losses) / len(model.val_losses)
    print(f"Test Loss for {num_layers} layers, {neurons_per_layer} neurons per layer: {avg_val_loss:.4f}")

    return avg_val_loss


def optimize_hyperparameters():
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

    print("Best Hyperparameters: ", study.best_params)
    print("Best Value (Test Loss): ", study.best_value)
    
optimize_hyperparameters()

[I 2024-11-30 17:36:33,722] A new study created in memory with name: no-name-ccf43d6f-47e1-44fe-add9-e73978fa8441
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/.venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/lap_simulation/F1NeuralNetworkExperimentsLogger/2024-11-30_17-31-07/checkpoints exists and is not empty.

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | model   | Sequential | 83.3 K | train
1 | loss_fn | L1Loss     | 0      | train
-----------------------------------------------
83.3 K    Trainable params
0         Non-trainable params
83.3 K    Total params
0.333     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/i551965/Documents/dev/Uni/DataMining/ie500-data-mining-group7/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 4 records. Best score: 95116.000. Signaling Trainer to stop.
Epoch 0, global step 1932: 'val_loss' was not in top 1
[I 2024-11-30 17:36:54,993] Trial 0 finished with value: 95161.70421829446 and parameters: {'num_layers': 3, 'neurons_per_layer': 197}. Best is trial 0 with value: 95161.70421829446.


Test Loss for 3 layers, 197 neurons per layer: 95161.7042
Best Hyperparameters:  {'num_layers': 3, 'neurons_per_layer': 197}
Best Value (Test Loss):  95161.70421829446


### Determine the optimal learning rate, loss function and optimizer

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Sample hyperparameters from the search space
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)  # Learning rate (log scale)
    
    # Choose optimizer: Adam, AdamW, or SGD
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'AdamW', 'SGD'])
    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam
    elif optimizer_name == 'AdamW':
        optimizer = torch.optim.AdamW
    else:
        optimizer = torch.optim.SGD

    # Choose loss function: L1Loss or MSELoss
    loss_fn_name = trial.suggest_categorical('loss_fn', ['L1Loss', 'MSELoss'])
    if loss_fn_name == 'L1Loss':
        loss_fn = nn.L1Loss()
    else:
        loss_fn = nn.MSELoss()

    # Initialize the model with the fixed architecture and selected loss function
    model = LaptimePredicionModel(lr=lr, loss_fn=loss_fn, optimizer=optimizer)
    
    # Setup the trainer
    trainer = pl.Trainer(train_dataloader, max_epochs=10)

    # Fit the model
    trainer.fit(model)

    # Calculate average test loss
    avg_val_loss = sum(model.val_losses) / len(model.val_losses)
    print(f"Test Loss for Optimizer: {optimizer_name}, Loss function: {loss_fn_name}, "
          f"Learning rate: {lr:.6f}: {avg_val_loss:.4f}")

    return avg_val_loss  # We aim to minimize the test loss

# Function to run the optimization process
def optimize_hyperparameters():
    # Create an Optuna study to minimize the test loss
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)  # Run 10 trials

    # Print the best hyperparameters found by Optuna
    print("Best Hyperparameters: ", study.best_params)
    print("Best Test Loss: ", study.best_value)

# Start the optimization process
optimize_hyperparameters()

### Final Training with best configuration

In [ ]:
model = LaptimePredicionModel()
trainer = pl.Trainer(
    callbacks=[early_stopping, checkpoint_callback],
    log_every_n_steps=10
)

trainer = pl.Trainer(logger=logger)
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

### Final Evaluation on test data

In [ ]:
trainer.test(model=model, dataloaders=test_dataloader)